In [1]:
import os  # for file handling
import cv2  # for image processing
import tensorflow as tf  # for dl
import numpy as np  # for numerical operations
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [2]:
mnist = tf.keras.datasets.mnist  # loading the data for (0-9) handwritten digits

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:40000]
y_train = y_train[:40000]
print(x_train.shape)
print(x_test.shape)

(40000, 28, 28)
(10000, 28, 28)


In [4]:
# normalising the data
x_train = tf.keras.utils.normalize(
    x_train, axis=1
)  # normalised along individual rows hence axis=1
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [5]:
# creating the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [6]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [7]:
# fitting the model
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1250/1250 [==============================] - 7s 4ms/step - loss: 0.2629 - accuracy: 0.9234
Epoch 2/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.1022 - accuracy: 0.9679
Epoch 3/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0657 - accuracy: 0.9792
Epoch 4/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0435 - accuracy: 0.9863
Epoch 5/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0359 - accuracy: 0.9877
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0278 - accuracy: 0.9910
Epoch 7/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0213 - accuracy: 0.9927
Epoch 8/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0189 - accuracy: 0.9935
Epoch 9/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0167 - accuracy: 0.9943
Epoch 10/10
1250/1250 [==============================] - 24s 19ms/step - loss: 0.0

In [8]:
model.save("numbers.model")

INFO:tensorflow:Assets written to: numbers.model\assets


INFO:tensorflow:Assets written to: numbers.model\assets


In [9]:
# evaluating the model
loss, accuracy = model.evaluate(x_test, y_test)
print(loss * 100)
print(accuracy * 100)

313/313 [==============================] - 6s 12ms/step - loss: 0.1434 - accuracy: 0.9713
14.343683421611786
97.13000059127808


In [10]:
model = tf.keras.models.load_model("numbers.model")


def predict_digit(image_path):
    try:
        img = cv2.imread(image_path)[:, :, 0]
        img = np.invert(np.array([img]))
        prediction = model.predict(img)
        return np.argmax(prediction)
    except:
        return None


def open_file_dialog():
    file_path = filedialog.askopenfilename(
        filetypes=[("Image files", "*.png *.jpg *.jpeg *.gif *.bmp")]
    )
    if file_path:
        digit = predict_digit(file_path)
        if digit is not None:
            result_label.config(text=f"The number is probably a {digit}")
            img = Image.open(file_path)
            # Increase image size by 2 times
            img = img.resize((img.width * 2, img.height * 2), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            image_label.config(image=img)
            image_label.image = img
            open_button.pack_forget()  # Hide the "Open Image" button
        else:
            result_label.config(text="Error")


# Create the main window with size 400x400
root = tk.Tk()
root.title("Digit Recognition")
root.geometry("400x400")

# Create and configure widgets
result_label = tk.Label(root, text="", font=("Helvetica", 16))
image_label = tk.Label(root)
open_button = tk.Button(root, text="Open Image", command=open_file_dialog)

# Layout widgets initially in the middle
result_label.pack(pady=50)
image_label.pack()
open_button.pack(pady=10)

root.mainloop()

1/1 [==============================] - 0s 373ms/step


C:\Users\parth\AppData\Local\Temp\ipykernel_14320\1436684425.py:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((img.width * 2, img.height * 2), Image.ANTIALIAS)
